<a href="https://colab.research.google.com/github/IzhbuldinaDaria2002/-/blob/main/%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

# Загрузка исходных файлов
df_orders = pd.read_csv('web_app_orders_traffic_all.csv')
df_app_sessions = pd.read_csv('app_traffic_all.csv')
# Объединение данных
# Извлекаем web-сессии (platform='web')
web_sessions = df_orders[df_orders['platform'] == 'web'][['date', 'country', 'channel_1', 'channel_2', 'sessions']].copy()
web_sessions.rename(columns={'sessions': 'web_sessions'}, inplace=True)
# Суммируем заказы и доход по обеим платформам (без учёта platform)
orders_revenue = df_orders.groupby(['date', 'country', 'channel_1', 'channel_2'], as_index=False).agg({
    'orders': 'sum',
    'revenue': 'sum'
})
# Берём app-сессии из второго файла
app_sessions = df_app_sessions[['date', 'country', 'channel_1', 'channel_2', 'sessions']].copy()
app_sessions.rename(columns={'sessions': 'app_sessions'}, inplace=True)

# Последовательно объединяем три части (outer join, чтобы не потерять данные)
merged = pd.merge(orders_revenue, web_sessions, on=['date', 'country', 'channel_1', 'channel_2'], how='outer')
merged = pd.merge(merged, app_sessions, on=['date', 'country', 'channel_1', 'channel_2'], how='outer')

# Заполняем пропуски нулями
merged.fillna({'web_sessions': 0, 'app_sessions': 0, 'orders': 0, 'revenue': 0}, inplace=True)

# Считаем общие сессии
merged['sessions'] = merged['web_sessions'] + merged['app_sessions']

# Сохраняем детальный датасет
merged.to_csv('merged_data.csv', index=False)
print("Создан файл merged_data.csv – детальные данные по дням и каналам.")

# Подготовка KPI (общие цифры за январь)

# Загружаем merged и преобразуем дату
df = pd.read_csv('merged_data.csv')
df['date'] = pd.to_datetime(df['date'])
df_jan = df[df['date'].dt.month == 1].copy()
df_jan['year'] = df_jan['date'].dt.year

# Агрегируем по годам
total = df_jan.groupby('year').agg({
    'sessions': 'sum',
    'orders': 'sum',
    'revenue': 'sum'
}).reset_index()

# Рассчитываем CR и AOV
total['cr'] = total['orders'] / total['sessions']
total['aov'] = total['revenue'] / total['orders']

# Разделяем на 2024 и 2025
total_2024 = total[total['year'] == 2024].iloc[0] if 2024 in total['year'].values else None
total_2025 = total[total['year'] == 2025].iloc[0] if 2025 in total['year'].values else None

if total_2024 is not None and total_2025 is not None:
    kpi_data = {
        'metric': ['Sessions', 'Orders', 'Revenue', 'CR', 'AOV'],
        'value_2025': [
            total_2025['sessions'],
            total_2025['orders'],
            total_2025['revenue'],
            total_2025['cr'],
            total_2025['aov']
        ],
        'delta_percent': [
            (total_2025['sessions'] - total_2024['sessions']) / total_2024['sessions'] * 100,
            (total_2025['orders'] - total_2024['orders']) / total_2024['orders'] * 100,
            (total_2025['revenue'] - total_2024['revenue']) / total_2024['revenue'] * 100,
            (total_2025['cr'] - total_2024['cr']) / total_2024['cr'] * 100,
            (total_2025['aov'] - total_2024['aov']) / total_2024['aov'] * 100
        ]
    }
    kpi_df = pd.DataFrame(kpi_data)
    kpi_df.to_csv('kpi_total.csv', index=False)
    print("Создан файл kpi_total.csv – общие KPI за январь 2025 и дельта.")
else:
    print("Внимание: не хватает данных за один из годов. KPI не рассчитаны.")

# Подготовка данных по каналам с дельтой ----

# Группируем по году, стране и каналам
grouped = df_jan.groupby(['year', 'country', 'channel_1', 'channel_2']).agg({
    'sessions': 'sum',
    'orders': 'sum',
    'revenue': 'sum'
}).reset_index()

# Добавляем CR и AOV
grouped['cr'] = grouped['orders'] / grouped['sessions']
grouped['aov'] = grouped['revenue'] / grouped['orders']

# Разделяем на 2024 и 2025
g2024 = grouped[grouped['year'] == 2024].copy()
g2025 = grouped[grouped['year'] == 2025].copy()

# Объединяем для расчёта дельты (outer join, чтобы учесть каналы, появившиеся только в одном году)
delta_channels = pd.merge(
    g2024, g2025,
    on=['country', 'channel_1', 'channel_2'],
    suffixes=('_2024', '_2025'),
    how='outer'
).fillna(0)

# Считаем дельту для каждой метрики (в процентах)
for metric in ['sessions', 'orders', 'revenue', 'cr', 'aov']:
    col_2024 = f'{metric}_2024'
    col_2025 = f'{metric}_2025'
    # Защита от деления на ноль
    delta_channels[f'delta_{metric}'] = delta_channels.apply(
        lambda row: (row[col_2025] - row[col_2024]) / row[col_2024] * 100 if row[col_2024] != 0 else 0,
        axis=1
    )

# Сохранение
delta_channels.to_csv('delta_by_channels.csv', index=False)
print("Создан файл delta_by_channels.csv – данные по каналам с дельтой.")

Создан файл merged_data.csv – детальные данные по дням и каналам.
Создан файл kpi_total.csv – общие KPI за январь 2025 и дельта.
Создан файл delta_by_channels.csv – данные по каналам с дельтой.
